<a href="https://colab.research.google.com/github/yuusei-2001/General-Theory-of-Intelligent-Systems-Programming/blob/main/%E5%AE%8C%E6%88%90YT_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#google colabo
#インストール
!pip install pulp
!pip install datetime
!pip install jpholiday

In [ ]:
#インポート
import pulp
import csv
import pandas as pd
import datetime
import jpholiday

In [ ]:
#休日の判定プログラム

year = input('年度を入力してください: ')
month = input('月を入力してください：')

year = int(year)
month = int(month)

def isBizDay(day):
    Date = datetime.date(year, month, day)
    if Date.weekday() >= 5 or jpholiday.is_holiday(Date):
        return 0
    else:
        return 1

if year % 4 == 0:
    months = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
else:
    months = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

days = months[month - 1]

# 日付の集合
D = list(range(1, days + 1))

# 曜日の判定
days_of_week = ["月", "火", "水", "木", "金", "土", "日"]
start_date = datetime.date(year, month, 1)

# 例として、5月1日が営業日かどうかを確認
print(days)

年度を入力してください: 2024
月を入力してください：6
30


In [ ]:
import pulp
number=11
# ナースと勤務区分の集合を定義
M = list(range(1, number))  # ナース10人
C = ['/','N', 'D', 'A']  # 休み, 早番, 遅番, 夜勤, 昼勤, 夜勤明け

# 勤務時間の定義
shift_hours = {
    'N': list(range(22, 24)) + list(range(0, 6)),  # 夜勤 (22:00-6:00)
    'D': range(8, 16),  # 昼勤
    'A': list(range(6, 24)) + list(range(0, 12))  # 夜勤明け(6:00-翌12:00)
}

#Q = ['AN','ME', 'MN', 'EM', 'EN', 'DN', 'DM']
Q = ['AN']

# ナースの勤務スケジュール変数を定義
x = pulp.LpVariable.dicts("x", (M, range(days+1), C), 0, 1, pulp.LpBinary)

# 問題の定義
problem = pulp.LpProblem("NurseSchedulingProblem", pulp.LpMinimize)
#problem = pulp.LpProblem("NurseSchedulingProblem", pulp.LpMaximize)
# 制約(1)夜勤（N）以外の日に夜勤明け（AN）が割り当てられないことを保証
for m in M:
    for d in D:
        if d == 1:  # 最初の日の場合、前日のデータはないため特別扱い
            problem += x[m][d]['A'] == 0
        else:
            problem += x[m][d]['A']  <= x[m][d - 1]['N']
# 制約(1)：夜勤の後は夜勤明けにする制約
for n in M:
    for d in range(days-1):
        problem += x[n][d]['N'] <= x[n][d+1]['A']

# 制約(2)：各ナースが一日に1つのシフトにのみ入る制約
for n in M:
    #for d in range(days):
    for d in D:
        problem += pulp.lpSum([x[n][d][c] for c in C]) == 1


# 制約(3): 休みを11回以上確保する
for m in M:
  problem += pulp.lpSum(x[m][d]['/'] for d in D) >= days - 20

# 制約(4)：勤務人数の制約（例: 各シフトに少なくとも1人のナースが必要）
for hour in range(24):
    for d in D:
        # 22:00 - 6:00 (夜勤)
        if hour in shift_hours['N']:
            problem += pulp.lpSum(x[m][d]['N'] for m in M) == 1
        # 8:00 - 16:00 (昼勤)
        if hour in shift_hours['D']:
            problem += pulp.lpSum(x[m][d]['D'] for m in M) == 6 if isBizDay(d)==1 else 2
        """#夜勤明け(6:00-翌12:00)
        if hour in shift_hours['A'] and d != 1:
            problem += pulp.lpSum(x[m][d]['A'] for m in M) == 1 if isBizDay(d) == 1 else 0
            """

# 制約(5): 禁止シフト

for m in M:
    for d in D[1:]:
        if d < days:  # 最後の日は次の日がないので除外
            for q in Q:

                if q== "AN":
                   problem += x[m][d]['N'] + x[m][d-1]['A'] <= 1
                """if q == 'ME':
                    problem += x[m][d]['E'] + x[m][d-1]['M'] <= 1
                if q == 'MN':
                    problem += x[m][d]['N'] + x[m][d-1]['M'] <= 1
                if q == 'EM':
                    problem += x[m][d]['M'] + x[m][d-1]['E'] <= 1
                if q == 'EN':
                    problem += x[m][d]['N'] + x[m][d-1]['E'] <= 1
                if q == 'DN':
                    problem += x[m] [d] ['N'] + x[m][d-1]['D'] <= 1
                if q == 'DM':
                    problem += x[m][d]['M'] + x[m][d-1]['D'] <= 1"""

#連続勤務四日まで
for m in M:
   for d in range(days)[4:]:
    problem += pulp.lpSum(x[m][d-h][c] for h in range(5) for c in C if c != '/') <= 4

# 制約(6): 前後が休みになる孤立勤務を避ける
for m in M:
    for q in Q[1:]:
        t = len(q) - 1
        for d in range(days)[t:]:
            problem += pulp.lpSum(x[m][d-t+h][q[h]] for h in range(t+1)) <= t
"""
# 制約(6)：前後が休みになる孤立勤務を避ける
for m in M:
    for d in D:
        if d > 1 and d < days:  # 最初と最後の日は前後の日が存在しないため除外
            # d日目が勤務日（休日以外）である場合
            work_day = pulp.lpSum(x[m][d][c] for c in C if c != '/')
            # d-1日目とd+1日目が休日である場合
            before_day_off = x[m][d-1]['/']
            after_day_off = x[m][d+1]['/']
            # 勤務日が孤立することを禁止する制約
            problem += work_day <= before_day_off + after_day_off
# 目的関数（例: ここでは単純に0に設定）
#problem += 0
"""

# 補助変数の定義
dev = pulp.LpVariable.dicts("dev", M, lowBound=0)  # 各ナースの勤務日数の偏差

# 平均勤務日数
total_workdays = pulp.lpSum(x[m][d][c] for m in M for d in D for c in C if c != '/')
average_workdays = total_workdays / len(M)

# 勤務日数と平均の差の制約
for m in M:
    problem += pulp.lpSum(x[m][d][c] for d in D for c in C if c != '/') - average_workdays <= dev[m]
    problem += average_workdays - pulp.lpSum(x[m][d][c] for d in D for c in C if c != '/') <= dev[m]

# 目的関数の設定：勤務日数の偏差の合計を最小化
problem += pulp.lpSum(dev[m] for m in M)


# 問題を解く
# ソルバーの実行
pulp.LpStatus[problem.solve()]






'Optimal'

In [ ]:
print(pulp.value(average_workdays))

20.000000000000014


In [ ]:
def check_constraints():
    valid = True
    messages = []

    # 各ナースが一日に1つのシフトにのみ入る制約のチェック
    for n in M:
        for d in D:
            if sum(x[n][d][c].varValue for c in C) != 1:
                valid = False
                messages.append(f"ナース{n}が日{d}に複数のシフトに入っています。")

    # 夜勤の後は夜勤明けにする制約のチェック
    for n in M:
        for d in D[:-1]:
            if x[n][d]['N'].varValue == 1 and x[n][d + 1]['A'].varValue != 1:
                valid = False
                messages.append(f"ナース{n}が夜勤後に夜勤明けになっていません（日{d}）。")

    # 休みを11回以上確保する制約のチェック
    for m in M:
        if sum(x[m][d]['/'].varValue for d in D) < days - 20:
            valid = False
            messages.append(f"ナース{m}が十分な休みを確保していません。")

    # 勤務人数の制約のチェック
    for hour in range(24):
        for d in D:
            """if hour in shift_hours['M'] and isBizDay(d) == 1 and sum(x[m][d]['M'].varValue for m in M) != 0:
                valid = False
                messages.append(f"営業日{d}の6:00 - 14:00に早番の人数が不足しています。")
            if hour in shift_hours['E'] and isBizDay(d) == 1 and sum(x[m][d]['E'].varValue for m in M) != 0:
                valid = False
                messages.append(f"営業日{d}の14:00 - 22:00に遅番の人数が不足しています。")
            if hour in shift_hours['N'] and sum(x[m][d]['N'].varValue for m in M) != 1:
                valid = False

                messages.append(f"日{d}の22:00 - 6:00に夜勤の人数が不足しています。")"""
            if hour in shift_hours['D'] and isBizDay(d) == 1 and sum(x[m][d]['D'].varValue for m in M) != 6:
                valid = False
                messages.append(f"営業日{d}の8:00 - 16:00に昼勤の人数が不足しています。")
            """
            if hour in shift_hours['A'] and d != 1 and isBizDay(d) == 1 and sum(x[m][d]['A'].varValue for m in M) != 1:
                valid = False
                messages.append(f"営業日{d}の夜勤明け(6:00-翌12:00)の人数が不足しています。")

                """

        # 禁止シフトの制約チェック
    for m in M:
        for d in D:
            if d < days:  # 最後の日は次の日がないので除外
                for q in Q:
                    shift_today = q[0]
                    shift_tomorrow = q[1]
                    if x[m][d][shift_today].varValue == 1 and x[m][d + 1][shift_tomorrow].varValue == 1:
                        valid = False
                        messages.append(f"ナース{m}が禁止されたシフトパターン{q}に従っています（日{d}から日{d+1}）。")
    for m in M:
        for d in range(1, days - 3):
            if sum(x[m][d + i][c].varValue for i in range(5) for c in C if c != '/') > 4:
                valid = False
                messages.append(f"ナース{m}が日{d}から日{d + 4}まで連続勤務が5日以上になっています。")

    # 孤立勤務の制約チェック
    for m in M:
        for d in range(2, days):
            if x[m][d]['/'].varValue == 0 and x[m][d - 1]['/'].varValue == 1 and x[m][d + 1]['/'].varValue == 1:
                valid = False
                messages.append(f"ナース{m}が日{d}で孤立勤務になっています。")

    return valid, messages

# 制約のチェックと結果の表示
valid, messages = check_constraints()
if valid:
    print("すべての制約が満たされています。")
else:
    print("以下の制約が満たされていません:")
    for message in messages:
        print(message)



以下の制約が満たされていません:
ナース9が夜勤後に夜勤明けになっていません（日29）。
ナース1が日15で孤立勤務になっています。
ナース1が日28で孤立勤務になっています。
ナース2が日21で孤立勤務になっています。
ナース3が日2で孤立勤務になっています。
ナース3が日10で孤立勤務になっています。
ナース4が日7で孤立勤務になっています。
ナース4が日22で孤立勤務になっています。
ナース5が日8で孤立勤務になっています。
ナース6が日22で孤立勤務になっています。
ナース6が日29で孤立勤務になっています。
ナース7が日9で孤立勤務になっています。
ナース8が日10で孤立勤務になっています。
ナース8が日17で孤立勤務になっています。
ナース8が日24で孤立勤務になっています。
ナース9が日14で孤立勤務になっています。
ナース9が日16で孤立勤務になっています。
ナース10が日7で孤立勤務になっています。


In [ ]:
# 行列形式の出力
daywork = [0]*32
nightwork = [0]*32
late_shift = [0]*32
early_shift = [0]*32
after_night = [0]*32
member = [0]*32
names = ["","佐藤", "鈴木", "高橋", "田中", "伊藤", "渡辺", "山本", "中村", "小林", "加藤","A","B","C","D","E"]
day_count = 0
member_count = 0
# ANSIエスケープコードを使用した色の設定
color_codes = {
    '/': '\033[91m',  # 休日の日付を赤にする
    'END': '\033[0m'  # リセット
}
print("ナーススケジュール:",year,"年",month,"月")
print("日付 : ", end="")
for d in D:
        #日付と曜日の表示
    day_of_week = days_of_week[(start_date + datetime.timedelta(days=d-1)).weekday()]
    if isBizDay(d):
        print(f"｜{d:2}({day_of_week})｜ ", end="")
    else:
        print(f"｜{color_codes['/']}{d:2}({day_of_week}){color_codes['END']}｜ ", end="")
print()
print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
for m in M:
    print(names[m],":", end=" ")
    for d in D:
        found = False
        for c in C:
            if pulp.value(x[m][d][c]) == 1:
                if c == '/':
                    print("｜ 休み ｜", end=" ")
                elif c == 'D':
                    print("｜ 日勤 ｜", end=" ")
                    #日勤人数カウント
                    day_count = daywork[d]
                    daywork[d] = day_count +1
                    day_count = 0
                    '''出勤人数カウント
                    member_count = member[d]
                    member[d] = member_count +1
                    member_count = 0'''

                elif c == 'N':
                    print("｜ 夜勤 ｜", end=" ")
                    #夜勤人数カウント
                    day_count = nightwork[d]
                    nightwork[d] = day_count +1
                    day_count = 0
                    '''出勤人数カウント
                    member_count = member[d]
                    member[d] = member_count +1
                    member_count = 0'''
                elif c == 'E':
                  print("｜ 遅番 ｜", end=" ")
                  #遅番人数のカウント
                  day_count = late_shift[d]
                  late_shift[d] = day_count + 1
                  day_count = 0
                elif c == 'M':
                  print("｜ 早番 ｜", end=" ")
                  #遅番人数のカウント
                  day_count = early_shift[d]
                  early_shift[d] = day_count + 1
                  day_count = 0
                elif c == 'A':
                  print("｜ 夜明 ｜", end=" ")
                  #夜勤明け人数のカウント
                  day_count = after_night[d]
                  after_night[d] = day_count + 1
                  day_count = 0
                found = True
                break
        if not found:
            print("-", end=" ")
    print()
print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
'''print("出勤人数:",end=" ")
for d in D:
  print("｜ ",member[d],"｜", end=" ")
print()'''
print("日勤 :",end=" ")
for d in D:
  print("｜  " ,daywork[d]," ｜", end=" ")
print()
print("夜勤 :",end=" ")
for d in D:
  print("｜  ",nightwork[d]," ｜", end=" ")
print()
print("遅番 :",end=" ")
for d in D:
  print("｜  ",late_shift[d]," ｜",end=" ")
print()
print("早番 :",end=" ")
for d in D:
  print("｜  ",early_shift[d]," ｜",end=" ")
print()
print("夜明 :",end=" ")
for d in D:
  print("｜  ",after_night[d]," ｜",end=" ")
print()

ナーススケジュール: 2024 年 6 月
日付 : ｜ 1(土)｜ ｜ 2(日)｜ ｜ 3(月)｜ ｜ 4(火)｜ ｜ 5(水)｜ ｜ 6(木)｜ ｜ 7(金)｜ ｜ 8(土)｜ ｜ 9(日)｜ ｜10(月)｜ ｜11(火)｜ ｜12(水)｜ ｜13(木)｜ ｜14(金)｜ ｜15(土)｜ ｜16(日)｜ ｜17(月)｜ ｜18(火)｜ ｜19(水)｜ ｜20(木)｜ ｜21(金)｜ ｜22(土)｜ ｜23(日)｜ ｜24(月)｜ ｜25(火)｜ ｜26(水)｜ ｜27(木)｜ ｜28(金)｜ ｜29(土)｜ ｜30(日)｜ 
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
佐藤 : ｜ 夜勤 ｜ ｜ 夜明 ｜ ｜ 日勤 ｜ ｜ 休み ｜ ｜ 日勤 ｜ ｜ 日勤 ｜ ｜ 日勤 ｜ ｜ 休み ｜ ｜ 休み ｜ ｜ 日勤 ｜ ｜ 日勤 ｜ ｜ 夜勤 ｜ ｜ 夜明 ｜ ｜ 休み ｜ ｜ 日勤 ｜ ｜ 休み ｜ ｜ 休み ｜ ｜ 夜勤 ｜ ｜ 夜明 ｜ ｜ 日勤 ｜ ｜ 日勤 ｜ ｜ 休み ｜ ｜ 日勤 ｜ ｜ 日勤 ｜ ｜ 日勤 ｜ ｜ 日勤 ｜ ｜ 休み ｜ ｜ 日勤 ｜ ｜ 休み ｜ ｜ 休み ｜ 
鈴木 : ｜ 休み ｜ ｜ 休み ｜ ｜ 夜勤 ｜ ｜ 夜明 ｜ ｜ 休み ｜ ｜ 日勤 ｜ ｜ 日勤 ｜ ｜ 日勤 ｜ ｜ 休み ｜ ｜ 日勤 ｜ ｜ 日勤 ｜ ｜ 休み ｜ ｜ 日勤 ｜ ｜ 日勤 ｜ ｜ 休み ｜ ｜ 日勤 ｜ ｜ 夜勤 ｜ ｜ 夜明 ｜ ｜ 日勤 ｜ ｜ 休み ｜ ｜ 日勤 ｜ ｜ 休み ｜ ｜ 夜勤 ｜ ｜ 夜明 ｜ ｜ 日勤 ｜ ｜ 休み ｜ ｜ 日勤 ｜ ｜ 日勤 ｜ ｜ 休み ｜ ｜ 日勤 ｜ 
高橋 : ｜ 休み ｜ 